In [ ]:
import pandas as pd
weather=pd.read_csv("/content/drive/MyDrive/3656805.csv",index_col="DATE")
print(weather)

In [ ]:
weather.apply(pd.isnull).sum()

In [ ]:
weather.apply(pd.isnull).sum()/weather.shape[0]

In [ ]:
del weather["STATION"]


In [ ]:
print(weather)

In [ ]:
del weather["NAME"]

In [ ]:
print(weather)

In [ ]:
weather[pd.isnull(weather["PRCP"])]

In [ ]:
weather["PRCP"]=weather["PRCP"].fillna(0)
weather=weather.fillna(method="ffill")


In [ ]:
print(weather)

In [ ]:
core_weather=weather["1973-01-06":]


In [ ]:
print(core_weather)

In [ ]:
core_weather.dtypes

In [ ]:
core_weather.index

In [ ]:
core_weather.index=pd.to_datetime(core_weather.index)

In [ ]:
core_weather.index

In [ ]:
core_weather.index.year

In [ ]:
core_weather[["TMAX","TMIN"]].plot()

In [ ]:
core_weather.index.year.value_counts().sort_index()

In [ ]:
core_weather["PRCP"].plot()

In [ ]:
core_weather.groupby(core_weather.index.year).sum()["PRCP"]

In [ ]:
core_weather["target"]=core_weather.shift(-1)["TMAX"]

In [ ]:
core_weather

In [ ]:
core_weather=core_weather.iloc[:-1,:].copy()

In [ ]:
core_weather

In [ ]:
from sklearn.linear_model import Ridge
reg=Ridge(alpha=.1)


In [ ]:
predictors=["PRCP","TAVG","TMAX","TMIN"]


In [ ]:
train=core_weather.loc[:"2021-12-16"]

In [ ]:
test=core_weather.loc["2022-01-01":]

In [ ]:
reg.fit(train[predictors],train["target"])

In [ ]:
predictions=reg.predict(test[predictors])

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(test["target"],predictions)

In [ ]:
combinations=pd.concat([test["target"],pd.Series(predictions,index=test.index)],axis=1)
combinations.columns=["Actual","Prediction"]

In [ ]:
combinations

In [ ]:
combinations.plot()

In [ ]:
reg.coef_

In [ ]:
def giveprediction(predictors,core_weather,reg):
  train=core_weather.loc[:"2021-12-16"]
  test=core_weather.loc["2022-01-01":]
  reg.fit(train[predictors],train["target"])
  predictions=reg.predict(test[predictors])
  error=mean_absolute_error(test["target"],predictions)
  combinations=pd.concat([test["target"],pd.Series(predictions,index=test.index)],axis=1)
  combinations.columns=["Actual","Prediction"]
  return error,combinations

In [ ]:
core_weather["month_max"]=core_weather["TMAX"].rolling(30).mean()

In [ ]:
core_weather

In [ ]:
core_weather["month_day_max"]=core_weather["month_max"]/core_weather["TMAX"]

In [ ]:
core_weather["max_min"]=core_weather["TMAX"]/core_weather["TMIN"]

In [ ]:
predictors=["PRCP","TAVG","TMAX","TMIN","month_day_max","max_min"]

In [ ]:
core_weather=core_weather.iloc[30::].copy()

In [ ]:
error,combined=giveprediction(predictors,core_weather,reg)

In [ ]:
error

In [ ]:
combined.plot()

In [ ]:
core_weather["monthly_avg"]=core_weather["TMAX"].groupby(core_weather.index.month).transform(lambda x:x.expanding(1).mean())

In [ ]:
core_weather

In [ ]:
core_weather["day_of_year_avg"]=core_weather["TMAX"].groupby(core_weather.index.day).transform(lambda x:x.expanding(1).mean())

In [ ]:
core_weather

In [ ]:
predictors=["PRCP","TAVG","TMAX","TMIN","month_day_max","max_min","monthly_avg","day_of_year_avg"]

In [ ]:
error,combined=giveprediction(predictors,core_weather,reg)


In [ ]:
error

In [ ]:
combined.plot()

In [ ]:
reg.coef_

In [ ]:
core_weather.corr()["target"]

In [ ]:
combined["diff"]=(combined["Actual"]-combined["Prediction"]).abs()

In [ ]:
combined.sort_values("diff",ascending=False).head()

In [ ]:
error

In [ ]:
combined